# Inspiration for my 3am coding sessions
https://youtu.be/1SB5lMyPmuk?si=7lrMy8keMWtSzojO
$$\partial_t u + u \partial_x u - \nu \partial_{xx} u = 0 $$

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import tensorflow as tf
import torch
from PINN import PhysicsInformedNN as pinn

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))


2024-04-01 11:40:49.643674: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-01 11:40:49.750306: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-01 11:40:49.750395: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-01 11:40:49.760764: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-01 11:40:49.795747: I tensorflow/core/platform/cpu_feature_guar

Num GPUs Available:  1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-04-01 11:40:52.484632: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:f3:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-01 11:40:52.487426: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:f3:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-01 11:40:52.487453: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:f3:00.0/numa_node
Your kernel may have been built without NUMA support.


In [15]:
exact_data = sp.io.loadmat('burgers_shock.mat')
exact_time = exact_data['t'].flatten()[:]
exact_x = exact_data['t'].flatten()[:]


class YouCanHaveCheeseBorger(torch.nn.Module):
  def __init__(self,number_inputs,number_outputs,number_of_hidden_layers,number_of_layers) -> None:
    super().__init__()
    activation_layer = torch.nn.Tanh
    self.fcs = torch.nn.Sequential(*[
      torch.nn.Linear(number_inputs,number_of_hidden_layers),
      activation_layer()
    ])
    self.fch = torch.nn.Sequential(*[
      torch.nn.Sequential(*[
        torch.nn.Linear(number_of_hidden_layers,number_of_hidden_layers),
        activation_layer() 
      ]) for _ in range(number_of_layers-1)
    ])
    self.fce = torch.nn.Linear(number_of_hidden_layers,number_outputs)

  def forward(self,t,x):
    x = self.fcs(torch.cat((t, x), dim=1))
    # x = self.fcs(t,x)
    # x = self.fch(t,x)
    for layer in self.fch:
      x = layer(x)
    x = self.fce(x)
    # x = self.fce(t,x)
    return x

In [16]:
borger = YouCanHaveCheeseBorger(2,2,32,3)
time_boundary = torch.tensor(exact_time,requires_grad=True).view(-1,1)
x_boundary = torch.tensor(exact_x,requires_grad=True).view(-1,1)
time_physics = torch.linspace(0,1,20,requires_grad=True).view(-1,1)
x_physics = torch.linspace(-1,1,20,requires_grad=True).view(-1,1)
time_test = torch.linspace(0,1,100).view(-1,1)
optimizer = torch.optim.Adam(borger.parameters(),lr=1e-4)
print(time_boundary[0,0])
print(x_boundary[:,0])
for i in range(1000):
  optimizer.zero_grad()
  u1 = borger(time_boundary, x_boundary)
  u2 = borger(time_boundary.repeat(len(x_boundary), 1), x_boundary.repeat(len(time_boundary), 1))
  u3 = borger(time_boundary.repeat(len(x_boundary), 1), x_boundary.repeat(len(time_boundary), 1).t())

  # u1 = borger(time_boundary[0,0],x_boundary[:,0])
  boundary_loss1 = ((u1-torch.sin(torch.pi*x_boundary))**2)
  # u2 = borger(time_boundary,x_boundary[0])
  # u3 = borger(time_boundary,x_boundary[-1])
  boundary_loss2 = ((u2-0)**2 + (u3-0)**2)
  u = borger(time_physics,x_physics)
  dudt = torch.autograd.grad(u,time_physics,torch.ones_like(u),create_graph=True)[0]
  dudx = torch.autograd.grad(u,x_physics,torch.ones_like(u),create_graph=True)[0]
  dudxx = torch.autograd.grad(dudx,x_physics,torch.ones_like(u),create_graph=True)[0]
  nu = 0.01/torch.pi
  physics_loss = torch.mean((dudt + u*dudx - nu*dudxx)**2)

  total_loss = boundary_loss1 + boundary_loss2 + physics_loss
  total_loss.backward()
  optimizer.step()

tensor(0., dtype=torch.float64, grad_fn=<SelectBackward0>)
tensor([0.0000, 0.0100, 0.0200, 0.0300, 0.0400, 0.0500, 0.0600, 0.0700, 0.0800,
        0.0900, 0.1000, 0.1100, 0.1200, 0.1300, 0.1400, 0.1500, 0.1600, 0.1700,
        0.1800, 0.1900, 0.2000, 0.2100, 0.2200, 0.2300, 0.2400, 0.2500, 0.2600,
        0.2700, 0.2800, 0.2900, 0.3000, 0.3100, 0.3200, 0.3300, 0.3400, 0.3500,
        0.3600, 0.3700, 0.3800, 0.3900, 0.4000, 0.4100, 0.4200, 0.4300, 0.4400,
        0.4500, 0.4600, 0.4700, 0.4800, 0.4900, 0.5000, 0.5100, 0.5200, 0.5300,
        0.5400, 0.5500, 0.5600, 0.5700, 0.5800, 0.5900, 0.6000, 0.6100, 0.6200,
        0.6300, 0.6400, 0.6500, 0.6600, 0.6700, 0.6800, 0.6900, 0.7000, 0.7100,
        0.7200, 0.7300, 0.7400, 0.7500, 0.7600, 0.7700, 0.7800, 0.7900, 0.8000,
        0.8100, 0.8200, 0.8300, 0.8400, 0.8500, 0.8600, 0.8700, 0.8800, 0.8900,
        0.9000, 0.9100, 0.9200, 0.9300, 0.9400, 0.9500, 0.9600, 0.9700, 0.9800,
        0.9900], dtype=torch.float64, grad_fn=<SelectBackward

RuntimeError: mat1 and mat2 must have the same dtype, but got Double and Float